In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
df=pd.read_csv('kidney_disease.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
columns=pd.read_csv('data_description.txt',sep='-')
columns

In [ ]:
columns=columns.reset_index()
columns

In [ ]:
columns.columns=['cols','abb_col_names']
columns

In [ ]:
df.head()

# Columns name change 

In [ ]:
df.columns=columns['abb_col_names'].values

In [ ]:
df.head()

In [ ]:
df.dtypes

# COnvert Some Data types

In [ ]:
def Convert_dtype(df,feature):
    df[feature]=pd.to_numeric(df[feature],errors='coerce')

In [ ]:
features=['packed cell volume','white blood cell count','red blood cell count']

for feature in features:
    Convert_dtype(df,feature)

In [ ]:
df.dtypes

# Now Remove Some Columns

In [ ]:
df.drop('id',axis=1,inplace=True)

# Now Caterigial data

In [ ]:
def Extract_Cat_Num(df):
    cat_col=[col for col in df.columns if df[col].dtype=='object']
    num_col=[col for col in df.columns if df[col].dtype != 'object']
    return cat_col,num_col

In [ ]:
cat_col,num_col=Extract_Cat_Num(df)

In [ ]:
cat_col

In [ ]:
num_col

# Check Dirtyness Of Data

In [ ]:
for col in cat_col:
    print('{} has {} values' .format(col,df[col].unique()))
    print('\n')

In [ ]:
df['diabetes mellitus'].replace(to_replace={'\tno':'no','\tyes':'yes'},inplace=True)


df['coronary artery disease']=df['coronary artery disease'].replace(to_replace={'\tno':'no'})

df['class']=df['class'].replace(to_replace='ckd\t',value='ckd')

In [ ]:
for col in cat_col:
    print('{} has {} values' .format(col,df[col].unique()))
    print('\n')

# Now Valuazation

In [ ]:
len(num_col)

In [ ]:
plt.figure(figsize=(30,20))

for i,feature in enumerate(num_col):
    plt.subplot(5,4,i+1) # i+1 by default i value is 0
    df[feature].hist()
    plt.title(feature)

# Now Lable Distribution of catergial data

In [ ]:
len(cat_col)

In [ ]:
plt.figure(figsize=(20,20))

for i,feature in enumerate(cat_col):
    plt.subplot(5,4,i+1)
    sns.countplot(df[feature])

# Now Target Data

In [ ]:
sns.countplot(df['class'])

# Now Check Corelate of my feature

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(20,16))
sns.heatmap(df.corr(),annot=True)

In [ ]:
df.groupby(['red blood cells','class'])['red blood cell count'].agg(['count','mean','median','min','max'])

In [ ]:
import plotly.express as px

In [ ]:
df.columns

In [ ]:
px.violin(df,y='red blood cell count', x= 'class',color='class')

In [ ]:
df.columns

# FInd Relationship b/w homogolbn and packed cell valume

px.scatter(df,x='haemoglobin',y='packed cell volume')

# Now Chorinc and no chorinc distribution

In [ ]:
grid=sns.FacetGrid(df,hue='class',aspect=3)
grid.map(sns.kdeplot,'red blood cell count')
grid.add_legend()

# Now Automate anaylise

In [ ]:
def violin(col):
    fig=px.violin(df,y=col,x='class',color='class',box=True)
    return fig.show()

In [ ]:
def scatter(col1,col2):
    fig=px.scatter(df,x=col1,y=col2,color='class')
    return fig.show()

In [ ]:
def kde_plot(feature):
    grid=sns.FacetGrid(df,hue='class',aspect=3)
    grid.map(sns.kdeplot,feature)
    grid.add_legend()

In [ ]:
kde_plot('red blood cell count')

In [ ]:
kde_plot('haemoglobin')

# Now see Relationship between features

In [ ]:
scatter('red blood cell count','packed cell volume')

In [ ]:
scatter('red blood cell count','haemoglobin')

In [ ]:
scatter('haemoglobin','packed cell volume')

In [ ]:
violin('red blood cell count')

In [ ]:
violin('packed cell volume')

In [ ]:
scatter('red blood cell count','albumin')

# Perfome EDA 

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum().sort_values()

# Fill missing value with Random Values

In [ ]:
def Random_Value_imputation(feature):
    random_sample=df[feature].dropna().sample(df[feature].isnull().sum())
    random_sample.index=df[df[feature].isnull()].index
    df.loc[df[feature].isnull(),feature]=random_sample

In [ ]:
df[num_col].isnull().sum()

In [ ]:
for col in num_col:
    Random_Value_imputation(col)

In [ ]:
df[num_col].isnull().sum()

In [ ]:
df[cat_col].isnull().sum()

In [ ]:
Random_Value_imputation('red blood cells')
Random_Value_imputation(' pus cell')

In [ ]:
df[cat_col].isnull().sum()

In [ ]:
def impute_mode(feature):
    mode=df[feature].mode()[0]
    df[feature]=df[feature].fillna(mode).fillna(mode)

In [ ]:
for col in cat_col:
    impute_mode(col)

In [ ]:
df[cat_col].isnull().sum()

In [ ]:
df.head()

# Now Fuature Encoding 


In [ ]:
for col in cat_col:
    print('{} has {} categories '.format(col,df[col].nunique()))

# Now Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
for col in cat_col:
    df[col]=le.fit_transform(df[col])

In [ ]:
df.head()

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
ind_col=[col for col in df.columns if col != 'class']
dep_col='class'

In [ ]:
X=df[ind_col]
y=df[dep_col]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
ordered_rank_features=SelectKBest(score_func=chi2,k=20)
ordered_feature=ordered_rank_features.fit(X,y)

In [ ]:
ordered_feature

In [ ]:
ordered_feature.scores_

In [ ]:
df_scores=pd.DataFrame(ordered_feature.scores_,columns=['Score'])

In [ ]:
df_scores

In [ ]:
dfcols=pd.DataFrame(X.columns)
dfcols

In [ ]:
feature_rank=pd.concat([dfcols,df_scores],axis=1)

In [ ]:
feature_rank

In [ ]:
feature_rank.describe()

In [ ]:
feature_rank.columns=['features','Score']
feature_rank

In [ ]:
feature_rank.nlargest(10,'Score') # Top 10 Features

In [ ]:
selected_col=feature_rank.nlargest(10,'Score')['features'].values

In [ ]:
selected_col

In [ ]:
X_new=df[selected_col]

In [ ]:
X_new

In [ ]:
len(X_new)

In [ ]:
X_new.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_new,y,random_state=0,test_size=0.25)

In [ ]:
print(X_train.shape)

In [ ]:
print(X_test.shape)

In [ ]:
y_train.value_counts()

# XG bost Model

In [ ]:
from xgboost import XGBClassifier

In [ ]:
XGBClassifier()

In [ ]:
# Hyper Tune

In [ ]:
params={
    'learning_rate':[0.05,0.20,0.25],
    'max_depth':[5,8,10],
    'min_child_weight':[1,3,5,7],
    'gamma':[0.0,0.1,0.2,0.4],
    'colsample_bytree':[0.3,0.4,0.7]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
classifer=XGBClassifier()

In [ ]:
random_search=RandomizedSearchCV(classifer,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [ ]:
random_search.fit(X_train,y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

In [ ]:
classifier=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=8,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
classifier.fit(X_train,y_train)

In [ ]:
y_pred=classifier.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
accuracy_score(y_test,y_pred)